In [1]:
import sys
sys.path.append('../src')

In [2]:
from covid import daily_snapshot
from pathlib import Path

import population as pop
import pandas as pd
import requests

In [3]:
pd.set_option('max_rows', 100) 

In [4]:
date = input('date in mm-dd-yyyy format')

date in mm-dd-yyyy format 04-01-2020


In [5]:
state = input('state, e.g. New Jersey')

state, e.g. New Jersey New York


### Quick glance at the data

In [6]:
df = daily_snapshot(date)
df.head()

,Unnamed: 0,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key
FIPS,,,,,,,,,,,,
45001.0,NaN,Abbeville,South Carolina,US,2020-04-01 21:58:49,34.223334,-82.461707,4,0,0,0,"Abbeville, South Carolina, US"
22001.0,NaN,Acadia,Louisiana,US,2020-04-01 21:58:49,30.295065,-92.414197,47,1,0,0,"Acadia, Louisiana, US"
51001.0,NaN,Accomack,Virginia,US,2020-04-01 21:58:49,37.767072,-75.632346,7,0,0,0,"Accomack, Virginia, US"
16001.0,NaN,Ada,Idaho,US,2020-04-01 21:58:49,43.452658,-116.241552,195,3,0,0,"Ada, Idaho, US"
19001.0,NaN,Adair,Iowa,US,2020-04-01 21:58:49,41.330756,-94.471059,1,0,0,0,"Adair, Iowa, US"


In [7]:
df[df.Country_Region=='US'].describe()

,Unnamed: 0,Lat,Long_,Confirmed,Deaths,Recovered,Active
count,0.0,2229.000000,2229.000000,2230.000000,2230.000000,2230.000000,2230.0
mean,NaN,37.255595,-88.693085,95.682511,2.133184,3.800000,0.0
std,NaN,7.067256,18.464507,1096.510006,27.897826,179.446928,0.0
min,NaN,0.000000,-159.596679,0.000000,0.000000,0.000000,0.0
25%,NaN,33.964725,-95.563511,2.000000,0.000000,0.000000,0.0
50%,NaN,37.871198,-87.787962,5.000000,0.000000,0.000000,0.0
75%,NaN,41.435647,-81.962311,23.000000,1.000000,0.000000,0.0
max,NaN,64.807262,145.673900,47439.000000,1139.000000,8474.000000,0.0


### Aggregate by state

In [8]:
state_totals = df[df.Country_Region=='US'].groupby(by='Province_State').sum()

In [9]:
state_totals.head()

,Unnamed: 0,Lat,Long_,Confirmed,Deaths,Recovered,Active
Province_State,,,,,,,
Alabama,0.0,1950.187952,-5119.386303,1060,27,0,0
Alaska,0.0,362.539360,-862.001024,132,3,0,0
Arizona,0.0,505.103399,-1671.360527,1530,29,0,0
Arkansas,0.0,1846.341710,-4900.768064,584,10,0,0
California,0.0,1873.911674,-6028.510306,9399,199,0,0


In [10]:
state_totals['death_rate?'] = state_totals['Deaths'] / state_totals['Confirmed'] * 100.0

In [11]:
state_totals.describe()

,Unnamed: 0,Lat,Long_,Confirmed,Deaths,Recovered,Active,death_rate?
count,58.0,58.000000,58.000000,58.000000,58.000000,58.000000,58.0,57.000000
mean,0.0,1431.771054,-3408.567003,3678.827586,82.017241,146.103448,0.0,2.308301
std,0.0,1180.539394,2951.908853,11324.251290,261.141589,1112.690712,0.0,2.219763
min,0.0,0.000000,-13764.875282,0.000000,0.000000,0.000000,0.0,0.000000
25%,0.0,483.170922,-5253.566369,290.250000,5.000000,0.000000,0.0,1.535836
50%,0.0,1189.846001,-3076.060812,728.500000,17.500000,0.000000,0.0,2.020057
75%,0.0,2368.468833,-864.637952,2840.750000,65.000000,0.000000,0.0,2.552022
max,0.0,4566.435219,145.673900,83948.000000,1941.000000,8474.000000,0.0,16.666667


In [12]:
state_covid = df[df.Province_State == state].copy()
state_covid['death_rate'] = 100 * state_covid['Deaths'] / state_covid['Confirmed']
state_columns=['Admin2', 'Confirmed', 'Deaths', 'Recovered', 'Active', 'death_rate' ]

In [13]:
sort_by = input('sort by one of Admin2, Confirmed, Deaths, death_rate')

sort by one of Admin2, Confirmed, Deaths, death_rate Confirmed


In [14]:
state_covid[state_columns].sort_values(by=sort_by, ascending=False)

,Admin2,Confirmed,Deaths,Recovered,Active,death_rate
FIPS,,,,,,
36061.0,New York City,47439,1139,0,0,2.400978
36119.0,Westchester,10683,25,0,0,0.234017
36059.0,Nassau,9554,76,0,0,0.795478
36103.0,Suffolk,7605,53,0,0,0.696910
36087.0,Rockland,3321,18,0,0,0.542005
36071.0,Orange,1756,19,0,0,1.082005
36029.0,Erie,582,11,0,0,1.890034
36027.0,Dutchess,547,4,0,0,0.731261
36055.0,Monroe,349,9,0,0,2.578797


## We need to locate county data to add per capita rates.